In [2]:
import yfinance as yf
import pandas as pd
import datetime

# Define the list of tickers for the selected equities
tickers = ['CRWD', 'NVDA', 'HD']

# Initialize an empty dictionary to store the data
data = {}

# Retrieve 2-minute interval data for the past 60 days
start_date = (datetime.datetime.now() - datetime.timedelta(days=59)).strftime('%Y-%m-%d')
end_date = datetime.datetime.now().strftime('%Y-%m-%d')

for ticker in tickers:
    print(f"Fetching data for {ticker}")
    try:
        stock_data = yf.download(ticker, start=start_date, end=end_date, interval='2m')
        data[ticker] = stock_data
        
        # Save each stock's data into a separate CSV file
        stock_data.to_csv(f"{ticker}_2min_data.csv")
        
    except Exception as e:
        print(f"Could not retrieve data for {ticker}: {e}")

print("Data retrieval and saving complete!")

Fetching data for CRWD


[*********************100%%**********************]  1 of 1 completed


Fetching data for NVDA


[*********************100%%**********************]  1 of 1 completed


Fetching data for HD


[*********************100%%**********************]  1 of 1 completed

Data retrieval and saving complete!


In [3]:
# Load the previously saved CRWD data
crwd_data = pd.read_csv("CRWD_2min_data.csv", index_col=0, parse_dates=True)

# Calculate the 20-minute and 200-minute SMAs
crwd_data['20min_SMA'] = crwd_data['Close'].rolling(window=10).mean()  # 20min SMA on a 2min interval is a 10-period SMA
crwd_data['200min_SMA'] = crwd_data['Close'].rolling(window=100).mean()  # 200min SMA on a 2min interval is a 100-period SMA

# Save the updated data with SMAs to the same CSV file or a new one
crwd_data.to_csv("CRWD_2min_data_with_SMA.csv")

print("CRWD data with 20min and 200min SMAs saved successfully!")

CRWD data with 20min and 200min SMAs saved successfully!
